In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pytz
import io
import base64

In [77]:
df = pd.read_csv("Play Store Data.csv")

In [78]:
df = df[df['Installs'] != 'Free']

In [79]:
df['Installs'] = df['Installs'].str.replace('[+,]', '', regex=True).astype(float)
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')
df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')

In [80]:
df.dropna(subset=['App', 'Category', 'Reviews', 'Installs', 'Last Updated'], inplace=True)

In [81]:
filtered_df = df[
    (~df['App'].str.lower().str.startswith(('x', 'y', 'z'))) &
    (df['Category'].str.upper().str.startswith(('E', 'C', 'B'))) &
    (df['Reviews'] > 500)
].copy()

In [82]:
filtered_df['Month'] = filtered_df['Last Updated'].dt.to_period('M').dt.to_timestamp()

In [83]:
grouped_df = filtered_df.groupby(['Month', 'Category'])['Installs'].sum().reset_index()

In [84]:
grouped_df['Prev_Installs'] = grouped_df.groupby('Category')['Installs'].shift(1)
grouped_df['Growth_%'] = ((grouped_df['Installs'] - grouped_df['Prev_Installs']) / grouped_df['Prev_Installs']) * 100

In [85]:
plt.figure(figsize=(14, 7))
sns.set(style="whitegrid")

for category in grouped_df['Category'].unique():
    cat_data = grouped_df[grouped_df['Category'] == category]
    plt.plot(cat_data['Month'], cat_data['Installs'], label=category)
    for i in range(1, len(cat_data)):
        if cat_data.iloc[i]['Growth_%'] > 20:
            plt.fill_between(
                [cat_data.iloc[i - 1]['Month'], cat_data.iloc[i]['Month']],
                [cat_data.iloc[i - 1]['Installs'], cat_data.iloc[i]['Installs']],
                color='orange', alpha=0.3
            )

plt.title('Monthly Install Trends by App Category (MoM Growth > 20%)')
plt.xlabel('Month')
plt.ylabel('Total Installs')
plt.legend(title='Category')
plt.tight_layout()
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
plt.close()
buf.seek(0)

0

In [86]:
image_base64 = base64.b64encode(buf.read()).decode("utf-8")
image_tag = f'<img src="data:image/png;base64,{image_base64}" alt="Install Trend Graph" style="width:90%;">'

html_output = f"""
<!DOCTYPE html>
<html>
<head>
    <title>App Installs Trend</title>
    <script>
        function isISTBetween6and9PM() {{
            let now = new Date();
            let istOffset = 330; // IST = UTC +5:30
            let utc = now.getTime() + now.getTimezoneOffset() * 60000;
            let istTime = new Date(utc + (60000 * istOffset));
            let hour = istTime.getHours();
            return hour >= 18 && hour < 21;
        }}

        window.onload = function () {{
            if (isISTBetween6and9PM()) {{
                document.getElementById("plot").style.display = "block";
            }} else {{
                document.getElementById("message").style.display = "block";
            }}
        }};
    </script>
</head>
<body>
    <h2 style="text-align:center;">Monthly App Installs by Category (MoM Growth > 20%)</h2>
    <p style="text-align:center;">Filtered: App name not starting with X/Y/Z, categories starting with B/C/E, reviews > 500</p>

    <div id="plot" style="display:none; text-align:center;">
        {image_tag}
    </div>

    <div id="message" style="display:none; text-align:center; font-size:20px; padding:50px;">
         This chart is visible only between 6 PM and 9 PM IST.
    </div>
</body>
</html>
"""

# Save to file
with open("task5.html", "w", encoding="utf-8") as f:
    f.write(html_output)

